In [1]:
import numpy as np
import pandas as pd
import re, json, csv, requests, time, glob, tqdm
from bs4 import BeautifulSoup
from selenium import webdriver

In [34]:
def scrape(html):
    soup = BeautifulSoup(html, "html.parser")
    json_data = json.loads(soup.find_all("script", type="application/ld+json")[-1].text)
    title = json_data['headline']
    date = json_data['datePublished']
    date_m = json_data['dateModified']
    genre = json_data.get('genre', [])
    keywords = json_data.get('keywords', [])
    article = soup.find('div', id="news_textbody").text
    url_normal = 'https:' + soup.find('meta', property="og:url").get('content').rsplit('https:')[-1]
    if soup.find_all('div', id="news_textmore") != []:
        for textmore in soup.find_all('div', id="news_textmore"):
            article += ('\n' + textmore.text)
    if soup.find_all('div', class_="news_add") != []:
        for newsadd in soup.find_all('div', class_="news_add"):
            if newsadd.h3 != None:
                newsadd.h3.extract()
            article += ('\n' + newsadd.text)
            
    return {
        'id':url_normal.split('/')[-1].split('.html')[0],
        'title':title,
        'article':article.strip(),
        'genre':genre,
        'keywords':keywords,
        'url':url_normal,
        'datePublished':date,
        'dateModified':date_m
    }

def get_link(start=0):
    notyet = []
    n_list = pd.read_json('nhk/nhkweb.json', encoding='utf-8')['url'].tolist()
    df_e = pd.read_json('nhk/nhkwebeasy.json', encoding='utf-8') 
    for i in df_e['url_normal'][start:]:
        if i not in n_list:
            notyet.append(i)
    return notyet

def js(dic):
    with open('nhkweb.json', 'r', encoding='utf-8') as f:
        data = json.load(f)
    with open('nhkweb.json', 'w', encoding='utf-8') as f:
        if dic not in data:
            data.append(dic)
        data = sorted(data, key=lambda x:x['id'])
        json.dump(data, f, indent=4, ensure_ascii=False)
        
easy = pd.read_json('nhkwebeasy.json')

In [35]:
htmls = ['https://www3.nhk.or.jp/news/html/20191117/k10012180511000.html']

In [38]:
htmls = get_link(670); htmls

ValueError: Unexpected character found when decoding 'null'

# scrape

In [14]:
driver = webdriver.Chrome()

In [23]:
for nhkurl in tqdm.tqdm(htmls[:-2]):
    driver.get(f'https://web.archive.org/web/2019*/{nhkurl}')
    time.sleep(10)
    html = str(driver.page_source.encode('utf-8'))
    snap = re.search(r'(between|1 time).*?<a href="(.+?)">', html)
    if not snap:
        print(nhkurl)
        continue
    driver.get('https://web.archive.org' + snap.group(2))
    time.sleep(30)
    html = driver.page_source.encode('utf-8')
    dic = scrape(html)
    js(dic)




  0%|          | 0/20 [00:00<?, ?it/s]


  5%|▌         | 1/20 [01:45<33:30, 105.83s/it]


 10%|█         | 2/20 [03:22<30:56, 103.15s/it]


 15%|█▌        | 3/20 [04:47<27:37, 97.49s/it]


 20%|██        | 4/20 [06:29<26:22, 98.90s/it]


 25%|██▌       | 5/20 [08:04<24:27, 97.82s/it]


 30%|███       | 6/20 [09:28<21:52, 93.72s/it]


 35%|███▌      | 7/20 [10:44<19:07, 88.28s/it]


 40%|████      | 8/20 [12:20<18:09, 90.78s/it]


 45%|████▌     | 9/20 [13:56<16:55, 92.36s/it]


 50%|█████     | 10/20 [15:18<14:52, 89.24s/it]


 55%|█████▌    | 11/20 [16:28<12:29, 83.27s/it]


 60%|██████    | 12/20 [17:58<11:22, 85.30s/it]


 65%|██████▌   | 13/20 [19:29<10:08, 86.97s/it]


 70%|███████   | 14/20 [20:38<08:11, 81.84s/it]


 75%|███████▌  | 15/20 [21:44<06:24, 76.93s/it]


 80%|████████  | 16/20 [22:59<05:05, 76.45s/it]


 85%|████████▌ | 17/20 [24:14<03:48, 76.05s/it]


 90%|█████████ | 18/20 [25:22<02:26, 73.44s/it]


 95%|█████████▌| 19/20 [26:28<01:11, 71.33s/it]


100%|████████

In [44]:
html = driver.page_source.encode('utf-8')
dic = scrape(html)
js(dic)

In [15]:
re.findall(r'1 time(.............................)', html)

['</strong>&nbsp;<a href="/web/']